In [15]:
# Leer ficheros de datos
import codecs
import pandas as pd
import pyarrow.parquet as pq
# Extraer información
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
# Auxiliares
import numpy as np
import logging
from typing import Union
import pyarrow as pa

In [16]:
df = pd.read_parquet('Datasets/PHA/PHA_Data.parquet', engine='pyarrow')

In [17]:
def clean_data(value): 
    # Si se trata de un valor en blanco sustituir a NaN 
    try: 
        if value in [None, 'nan']: 
            return np.nan 
         
        return value 
 
    except Exception as e: 
        return value 
     
# Iterar sobre todas las columnas y comprobar sus valores 
columns = len(df.columns) 
 
for column in df.columns: 
    # Utilizar compresion de listas y aplicar el filtrado de valores 
    values = [clean_data(value) for value in df[column]] 
    df[column] = values 
    columns -= 1 
    print(f'[+] REMAINING_COLUMNS\t{columns}\t', end='\r') 
     
df

,c1,c6,c7,c8,c10,c132,c134,c136,c138,c141,...,c150,c154,c156,c158,c161,c163,c183,c191,c229,c230
0,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,NaN,NaN
2,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,NaN,NaN
3,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,NaN,NaN
4,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214882,I,2022,2,6,0930,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,19470720.0,NaN
214883,I,2022,2,6,1710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,19700731.0,NaN
214884,I,2022,1,31,0832,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,19810601.0,19980409.0
214885,I,2022,1,7,0407,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,19520606.0,NaN


In [19]:
url_legend = 'https://av-info.faa.gov/data/AID/Afilelayout.txt'
# Leemos el contenido de la leyenda
with urlopen(url_legend) as content:
    soup = BeautifulSoup(content, "html.parser")
    soup_lines = str(soup).split('\r\n')

# Transformamos la respuesta en un diccionario con el nombre de la columna y la descripción
legend_df = {"Column_name": [], "Description": []}
# Recorrer las lineas con datos de lineas. Se salta las dos primeras y las tres ultimas
# filas por no tener datos relevantes de la leyenda
for line in soup_lines[2:-3]:
    # Se extrae los 5 primeros caracteres para conocer el nombre de la columna
    legend_df["Column_name"].append(line[0:5].strip())
    # Se extrae la descripcion, esta comienza en la posicion 53
    legend_df["Description"].append(line[53:].strip())
    
# Convertir el diccionario en pandas.Dataframe
legend_df = pd.DataFrame.from_dict(data=legend_df)
legend_df

,Column_name,Description
0,c5,Unique control number used to relate to AID_MA...
1,c1,Type of Event
2,c2,FAR part number
3,c3,Form on which the latest data was received.
4,c4,Agency conducting investigation.
...,...,...
174,c163,2nd Additional cause factor text
175,c183,Supporting Factor Text ...
176,c191,Supporting cause factor B text. O...
177,c229,Date of Birth of PIC


In [25]:
cabeceras = list(df.columns)
#unused_columns = [5, 3, 2, 4, 9, 75, 140, 139, 203, 204, 214, 790, 26, 37, 15, 16, 17,18, 19, 143, 205, 206, 207, 208, 210, 43, 129, 124, 125, 77]
#unused_columns = [f'c{idx}' for idx in unused_columns]

for col in cabeceras:
    description = legend_df[legend_df["Column_name"] == col]["Description"].values[0]
    print(f'[+] Descripción columnas\t{col}\t{description}')

[+] Descripción columnas	c1	Type of Event
[+] Descripción columnas	c6	Year the accident/incident happened.
[+] Descripción columnas	c7	Month the accident/incident happened.
[+] Descripción columnas	c8	Day the accident/incident happened.
[+] Descripción columnas	c10	Local time of the accident/incident.
[+] Descripción columnas	c132	First remedial action area code
[+] Descripción columnas	c134	First remedial action taken code
[+] Descripción columnas	c136	Second remedial action area code
[+] Descripción columnas	c138	Second remedial action taken code
[+] Descripción columnas	c141	Method of notification
[+] Descripción columnas	c144	Code for Aircraft Design
[+] Descripción columnas	c145	Code for Weight  Class
[+] Descripción columnas	c147	Wing information of the aircraft involved in the investigation.
[+] Descripción columnas	c149	Powered, Nonpowered, Optional
[+] Descripción columnas	c151	Number of engines on aircraft
[+] Descripción columnas	c152	Code representing typical engine design 

# Selección de caracteristicas:


In [ ]:
Descripción columnas
c1 Tipo de Evento
c6 Año en que ocurrió el accidente/incidente.
Mes en que ocurrió el accidente/incidente.
c8 Día en que ocurrió el accidente/incidente.
c10 Hora local del accidente/incidente.
c132 Código de área de primera acción correctiva
c134 Código primera acción correctiva tomada
c136 Código de área de la segunda acción correctiva
c138 Código segunda acción correctiva tomada
c141 Forma de notificación
c160 Código de factor de causa adicional
c162 Código factor causa adicional 2
c233 Bandera para indicar si el piloto murió.
c234 Bandera para indicar si el segundo piloto murió.
c23 Marca de la aeronave
c24 Modelo de la aeronave
c11 Región del lugar del accidente/incidente.
c12 Oficina distrital del lugar del accidente/incidente.
c14 Ciudad del accidente/lugar del incidente.
Coordenadas de latitud del accidente/incidente
c21 Coordenadas de longitud del accidente/incidente
c102 Tipo primario de código de vuelo
c104 Tipo secundario de código de vuelo
c106 Código de condición primaria de vuelo
c108 Código de condición de vuelo secundario
c110 Código estado luz
c114 Visibilidad código
c115 Código de restricción de visibilidad.
c117 Código de condición de la pista
c118 Código de condición de frenado
c61 Incluye tripulantes y pasajeros.
c62 Número de pasajeros a bordo
c65 Número de tripulantes de vuelo a bordo
c67 Número de tripulantes de vuelo heridos
c68 Número de tripulantes de cabina a bordo
c41 Código del tipo de certificado del piloto al mando
c45 Código de habilitación del piloto al mando
c47 Cualquier otro código de habilitación del piloto al mando.
c49 Código de habilitación del piloto al mando.
c50 Edad del piloto al mando
c52 Profesión


c53 Número total de horas que el piloto ha volado en la marca y modelo específico de la aeronave.
c54 Número total de horas en los últimos 90 días que el piloto ha volado esta marca y modelo
c55 Número total de horas en los últimos 90 días que ha volado el piloto.
c56 Número total de horas que ha volado el piloto.
c57 Código de la región de residencia del piloto al mando
c58 Código de oficina de distrito de residencia del piloto al mando
c59 Estado de residencia del piloto al mando
c119 Comentarios parciales sobre informes y donde se necesita conservar espacio.
c120 Nombre del propietario/operador de la aeronave
c121 Estado del propietario/operador
c122 Código designador del operador aéreo (OPCD)
c126 Número de certificado de la agencia aérea
c127 Región del operador aéreo
c128 Oficina distrital del operador aéreo
c130 Descripción del plan de vuelo
c78 Código de factor de causa primaria
c80 Código factor contribuyente
c82 Código de personas involucradas
c84 Código de factor de apoyo TÉCNICA
c86 Código de factor causa secundaria
c88 Segundo caso contribuyente código f
c90 Código de segundas personas involucradas
c92 Código del segundo factor de apoyo. TÉCNICO
c94 Código tipo de accidente
c96 Fase del código de vuelo
c40 Texto tipo certificado del piloto al mando
c44 Calificación descripción del piloto al mando
c46 Cualquier otra habilitación descripción del piloto al mando
c48 Descripción de la cualificación del piloto al mando
c51 Descripción de la profesión del piloto al mando
c81 Personas involucradas texto
c89 Segundas personas involucradas texto
c91 Texto del segundo factor de apoyo TÉCNICA
c93 Tipo de accidente o incidente texto
c229 Fecha de Nacimiento del PIC
c230 Fecha de nacimiento del segundo Piloto







